# Capstone Project 1

In [1]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Capstone Project 2

In [31]:
#read data into pandas dataframe
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [32]:
#remove all boroughs that are not assigned
df = df[df['Borough'] != 'Not assigned']

In [33]:
#check for duplicate postal codes in data
#this returns False, meaning there are none, it appears the wikipedia page has been updated and has been formated to match the description of the data formating in this project
np.any(df['Postal Code'].value_counts().values > 1)

False

In [35]:
#check if any Boroughs have "Not assigned" Neighborhoods, if returns 0 then there are none
#it appears the wikipedia page has been updated and has been formated to match the description of the data formating in this project
df[df['Neighborhood'] == 'Not assigned'].shape[0]

0

In [39]:
#find number of rows in data
print('Number of Rows in Data:',df.shape[0])

Number of Rows in Data: 103


# Capstone Project 3

In [46]:
#read in the lat/log data and merge it with the postal code/borough/neighborhood data
df = df.merge(pd.read_csv('https://cocl.us/Geospatial_data'), on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
